In [1]:
import pandas as pd
import numpy as np
import os
import glob
import childespy
import os

R[write to console]: Installing package into ‘/home/stephan/R/x86_64-pc-linux-gnu-library/4.2’
(as ‘lib’ is unspecified)



Reinstalling childesr version 0.2.1


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/childesr_0.2.3.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 22865 bytes (22 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to conso

In [2]:
# get the table of Providence Utterances
prov_utts = childespy.get_utterances(corpus = "Providence")

R[write to console]: Using current database version: '2021.1'.

R[write to console]: Getting data from 7 children in 1 corpus ...



In [3]:
prov_transcripts = childespy.get_transcripts(corpus = "Providence")

R[write to console]: Using current database version: '2021.1'.



In [4]:
prov_transcripts['filename_identifier'] = [os.path.basename(x).replace('.xml','') for x in prov_transcripts['filename']]

In [5]:
prov_utts_with_filenames = prov_utts.merge(prov_transcripts[['transcript_id','filename_identifier']], on=['transcript_id'])

In [6]:
prov_utts_with_filenames['media_duration'] = prov_utts_with_filenames['media_end'] - prov_utts_with_filenames['media_start'] 

# Sample Utterances

In [7]:
def sample_utterances(prov_utts_with_filenames, num_transcripts, num_utts_per_transcript):

    print(prov_utts_with_filenames.shape)
    prov_utts_with_filenames['num_words'] = [len(x.split(' ')) for x in prov_utts_with_filenames['gloss']]
    prov_utts_with_filenames['contains_xxx'] = prov_utts_with_filenames['gloss'].str.contains('xxx')
    prov_utts_with_filenames['contains_yyy'] = prov_utts_with_filenames['gloss'].str.contains('yyy')
    
    usable_utts = prov_utts_with_filenames.loc[
             ~prov_utts_with_filenames.contains_xxx & ~prov_utts_with_filenames.contains_yyy
    ]
    allowable_speakers = ('CHI','MOT')
    usable_utts = usable_utts.loc[usable_utts.num_words >= 2]
    usable_utts = usable_utts.loc[usable_utts.speaker_code.isin(allowable_speakers)]
    usable_utts = usable_utts.loc[~np.isnan(usable_utts.media_duration)]
    print(usable_utts.shape)
    
    chi_utts = usable_utts.loc[usable_utts.speaker_code == 'CHI']
    mot_utts = usable_utts.loc[usable_utts.speaker_code == 'MOT']
    
    chi_utts_by_transcript = [v for k, v in chi_utts.groupby(['filename_identifier'])]
    mot_utts_by_transcript = [v for k, v in mot_utts.groupby(['filename_identifier'])]
    
    np.random.seed(0)
    # Choose transcripts
    chi_transcript_choices = np.random.choice(range(len(chi_utts_by_transcript)), size=num_transcripts, replace=False)
    mot_transcript_choices = np.random.choice(range(len(mot_utts_by_transcript)), size=num_transcripts, replace=False)
    
    # Within each transcript, choose utterances
    sampled_utts = []
    for chi_transcript_choice in chi_transcript_choices:
        possible_utts = chi_utts_by_transcript[chi_transcript_choice]
        sampled_utt = np.random.choice(range(len(possible_utts)), size=num_utts_per_transcript, replace=False)[0]
        sampled_utts.append(possible_utts.iloc[sampled_utt].id)
        
    for mot_transcript_choice in mot_transcript_choices:
        possible_utts = mot_utts_by_transcript[mot_transcript_choice]
        sampled_utt = np.random.choice(range(len(possible_utts)), size=num_utts_per_transcript, replace=False)[0]        
        sampled_utts.append(possible_utts.iloc[sampled_utt].id)
    
    return(sampled_utts)    

In [8]:
utt_indices_to_score = sample_utterances(prov_utts_with_filenames, 20, 1)

(460061, 29)
(195144, 32)


/tmp/ipykernel_2743583/400814402.py:20: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  chi_utts_by_transcript = [v for k, v in chi_utts.groupby(['filename_identifier'])]
/tmp/ipykernel_2743583/400814402.py:21: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  mot_utts_by_transcript = [v for k, v in mot_utts.groupby(['filename_identifier'])]


In [9]:
utts_to_score = prov_utts_with_filenames.loc[prov_utts_with_filenames.id.isin(utt_indices_to_score)]

In [10]:
utts_to_score.head(5)

,id,gloss,stem,actual_phonology,model_phonology,type,language,num_morphemes,num_tokens,utterance_order,...,collection_id,corpus_id,speaker_id,target_child_id,transcript_id,filename_identifier,media_duration,num_words,contains_xxx,contains_yyy
18000,17278081,ding dong,ding dong,,,declarative,eng,2,2,416,...,21,336,23473,23471,43232,020411,1.998,2,False,False
44751,17323545,would you like to hear promised land,will you like to hear promise land,,,question,eng,9,7,1653,...,21,336,23473,23471,43247,021108,6.259,7,False,False
77965,17342025,I too,I too,ʌ tu,aɪ tuː,declarative,eng,2,2,1186,...,21,336,23487,23487,43268,010218,0.963,2,False,False
92097,17359147,tangled up moo,tangle up moo,ɛŋə ʌp mu,tæŋɡəld ʌp muː,declarative,eng,4,3,789,...,21,336,23487,23487,43275,010620,4.654,3,False,False
96534,17358854,over the bumps,over the bump,,,declarative,eng,4,3,454,...,21,336,23488,23487,43288,020007,1.790,3,False,False


# Extract WAVs from MP4s For Sampled Utterances

In [11]:
def extract_wav_from_mp4(base_media_path, utterance_record, base_output_folder):        
    mp4_filename = os.path.join(
        base_media_path,
        utterance_record['target_child_name'],
        utterance_record['filename_identifier']+'.mp4')
        
    output_folder = base_output_folder    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)        
    output_file = os.path.join(output_folder, str(utterance_record['id'])+'.wav')    
    command = "ffmpeg -hide_banner -loglevel error -i "+mp4_filename+" -ss "+str(utterance_record['media_start'])+  " -t "+str(utterance_record['media_duration'])+" -y -ac 1 -q:a 0 -map a -b:a 16k "+output_file
    print(command)
    os.system(command)
    
    if os.path.exists(output_file):
        return(output_file)
    else: 
        return('Error extracting WAV')

In [12]:
%%capture
base_media_path = '/hd1/Providence_videos/media.talkbank.org/phon/Eng-NA/Providence/'
utterance_records = utts_to_score.to_dict('records')
base_output_folder = 'extracted_wavs'

utts_to_score['output_wav'] = [extract_wav_from_mp4(base_media_path, x, base_output_folder) for x in utterance_records]

/hd1/Providence_videos/media.talkbank.org/phon/Eng-NA/Providence/Ethan/010218.mp4: No such file or directory
/hd1/Providence_videos/media.talkbank.org/phon/Eng-NA/Providence/Ethan/010620.mp4: No such file or directory
/hd1/Providence_videos/media.talkbank.org/phon/Eng-NA/Providence/Ethan/020007.mp4: No such file or directory
/hd1/Providence_videos/media.talkbank.org/phon/Eng-NA/Providence/Ethan/020709.mp4: No such file or directory
/hd1/Providence_videos/media.talkbank.org/phon/Eng-NA/Providence/Ethan/021101.mp4: No such file or directory
/hd1/Providence_videos/media.talkbank.org/phon/Eng-NA/Providence/Ethan/021018.mp4: No such file or directory


In [13]:
final_utts_to_score = utts_to_score.loc[utts_to_score.output_wav != 'Error extracting WAV']
final_utts_to_score

,id,gloss,stem,actual_phonology,model_phonology,type,language,num_morphemes,num_tokens,utterance_order,...,corpus_id,speaker_id,target_child_id,transcript_id,filename_identifier,media_duration,num_words,contains_xxx,contains_yyy,output_wav
18000,17278081,ding dong,ding dong,,,declarative,eng,2,2,416,...,336,23473,23471,43232,020411,1.998,2,False,False,extracted_wavs/17278081.wav
44751,17323545,would you like to hear promised land,will you like to hear promise land,,,question,eng,9,7,1653,...,336,23473,23471,43247,021108,6.259,7,False,False,extracted_wavs/17323545.wav
151803,17425579,boots boots,boot boot,bu bu,buːts buːts,declarative,eng,4,2,1629,...,336,23495,23495,43324,010719,3.345,2,False,False,extracted_wavs/17425579.wav
160848,17441440,I don't think you've ever had eggplant but Mom...,I do think you ever have but Mommy love,,,declarative,eng,13,11,1746,...,336,23497,23495,43335,020011,3.487,11,False,False,extracted_wavs/17441440.wav
179481,17437309,come here angel,come here angel,,,declarative,eng,3,3,315,...,336,23497,23495,43344,020227,0.777,3,False,False,extracted_wavs/17437309.wav
197711,17457797,was it,be it,wʌz ɪt,wɑz ɪt,question,eng,4,2,180,...,336,23495,23495,43364,020813,1.198,2,False,False,extracted_wavs/17457797.wav
200312,17458412,we did it,we do it,,,imperative_emphatic,eng,4,3,207,...,336,23497,23495,43361,020721,2.940,3,False,False,extracted_wavs/17458412.wav
206192,17479586,red red,red red,rɛː rɛːd,ɹɛd ɹɛd,declarative,eng,2,2,1408,...,336,23495,23495,43366,020904,9.045,2,False,False,extracted_wavs/17479586.wav
206957,17466565,um the pink one,the pink one,ʌm də pɪŋk wə,ʌm ðə pɪŋk wʌn,declarative,eng,3,4,192,...,336,23495,23495,43367,020909,5.157,4,False,False,extracted_wavs/17466565.wav
207669,17476426,there it is,there it be,,,declarative,eng,4,3,904,...,336,23497,23495,43367,020909,2.040,3,False,False,extracted_wavs/17476426.wav


# Recognize All and Compute WER By Speaker

In [14]:
import librosa
import nemo.collections.asr as nemo_asr
from IPython.display import Audio, display
import numpy as np
import pandas as pd
import gzip
import os, shutil, wget
import ctc_decoders
import copy
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import cdl_asr
import torchmetrics

[NeMo W 2023-03-29 14:49:15 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-03-29 14:49:16 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [36]:
import imp
imp.reload(cdl_asr)

<module 'cdl_asr' from '/home/stephan/notebooks/cdl-asr/cdl_asr.py'>

### With Neural Rescoring

In [46]:
# load all models
#lm_path = '/home/stephan/notebooks/child-directed-listening/output/ngram/childes_chi_sep_punct.LM'
lm_path = 'lowercase_3-gram.pruned.1e-7.arpa'
asr_model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name='QuartzNet15x5Base-En', strict=False)
# use BERT to rescore the hypotheses
bert_tokenizer = BertTokenizer.from_pretrained('/home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/train/n=5000/Providence_all_no_tagsxBERT')
bertMaskedLM = BertForMaskedLM.from_pretrained('/home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/train/n=5000/Providence_all_no_tagsxBERT')
bertMaskedLM.eval()
bertMaskedLM.to('cuda')  # if you have gpu
vocab =  np.array(bert_tokenizer.convert_ids_to_tokens(range(bert_tokenizer.vocab_size)))

[NeMo I 2023-03-29 17:01:32 cloud:56] Found existing object /home/stephan/.cache/torch/NeMo/NeMo_1.16.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.
[NeMo I 2023-03-29 17:01:32 cloud:62] Re-using file from: /home/stephan/.cache/torch/NeMo/NeMo_1.16.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo
[NeMo I 2023-03-29 17:01:32 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2023-03-29 17:01:32 features:286] PADDING: 16
[NeMo I 2023-03-29 17:01:33 audio_preprocessing:513] Numba CUDA SpecAugment kernel is being used
[NeMo I 2023-03-29 17:01:33 save_restore_connector:247] Model EncDecCTCModel was successfully restored from /home/stephan/.cache/torch/NeMo/NeMo_1.16.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.


In [47]:
def get_top_transcription(filepath, rescore):
    candidates = cdl_asr.transcribe_with_neural_rescoring([filepath],
        lm_path,
        asr_model,
        bertMaskedLM,
        vocab,
        bert_tokenizer,
        num_hypotheses=64,
        alpha=.8,
        rescore = rescore)    
    return candidates.iloc[0].hypothesis

In [48]:
%%time
final_utts_to_score['neural_transcription'] = [get_top_transcription(x, True) for x in final_utts_to_score.output_wav]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcript: "tink o down"


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Loading the LM will be faster if you build a binary file.
Reading lowercase_3-gram.pruned.1e-7.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


Best hypothesis after ngram: did


...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Loading the LM will be faster if you build a binary file.
Reading lowercase_3-gram.pruned.1e-7.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


Best hypothesis after ngram: hang


[NeMo W 2023-03-29 17:09:23 nemo_logging:349] /home/stephan/notebooks/cdl-asr/cdl_asr.py:135: SettingWithCopyWarning: 
    A value is trying to be set on a copy of a slice from a DataFrame.
    Try using .loc[row_indexer,col_indexer] = value instead
    
    See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
      all_hypotheses['bert_prob'] = [compute_prob_for_all_masks(hypothesis, bertMaskedLM, vocab, tokenizer, return_type='prob') for hypothesis in all_hypotheses.hypothesis]
    
[NeMo W 2023-03-29 17:09:23 nemo_logging:349] /home/stephan/notebooks/cdl-asr/cdl_asr.py:148: SettingWithCopyWarning: 
    A value is trying to be set on a copy of a slice from a DataFrame.
    Try using .loc[row_indexer,col_indexer] = value instead
    
    See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
      return(rescored_hypo

CPU times: user 7min 16s, sys: 47.5 s, total: 8min 3s
Wall time: 7min 48s


In [49]:
final_utts_to_score['neural_WER'] = [torchmetrics.functional.word_error_rate(x['neural_transcription'], 
    x['gloss']).item() for x in final_utts_to_score.to_dict('records')]

[NeMo W 2023-03-29 17:09:23 nemo_logging:349] /tmp/ipykernel_2743583/2749184977.py:1: SettingWithCopyWarning: 
    A value is trying to be set on a copy of a slice from a DataFrame.
    Try using .loc[row_indexer,col_indexer] = value instead
    
    See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
      final_utts_to_score['neural_WER'] = [torchmetrics.functional.word_error_rate(x['neural_transcription'],
    


In [50]:
final_utts_to_score['neural_WER']

18000     1.000000
44751     0.285714
151803    1.000000
160848    0.818182
179481    1.000000
197711    1.000000
200312    1.000000
206192    1.000000
206957    0.500000
207669    0.000000
215409    1.000000
217071    1.000000
226738    1.000000
237362    1.000000
252153    2.000000
254880    0.500000
258662    0.666667
260876    1.000000
266449    1.000000
277281    1.000000
288426    1.000000
289008    0.875000
296020    1.000000
297976    1.000000
311922    1.000000
333358    0.666667
335434    1.000000
341692    1.000000
342514    1.000000
364923    1.000000
416731    1.000000
430387    0.571429
439456    0.800000
457869    1.000000
Name: neural_WER, dtype: float64

### Without Neural Rescoring

In [51]:
final_utts_to_score['trigram_transcription'] = [get_top_transcription(x, False) for x in final_utts_to_score.output_wav]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcript: "tink o down"


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Loading the LM will be faster if you build a binary file.
Reading lowercase_3-gram.pruned.1e-7.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


Best hypothesis after ngram: did


...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcript: "han mon"


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Loading the LM will be faster if you build a binary file.
Reading lowercase_3-gram.pruned.1e-7.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
[NeMo W 2023-03-29 17:16:16 nemo_logging:349] /tmp/ipykernel_2743583/1811969063.py:1: SettingWithCopyWarning: 
    A value is trying to be set on a copy of a slice from a DataFrame.
    Try using .loc[row_indexer,col_indexer] = value instead
    
    See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
      final_utts_to_score['trigram_transcription'] = [get_top_transcription(x, False) for x in final_utts_to_score.output_wav]
    


Best hypothesis after ngram: hang


In [53]:
final_utts_to_score['trigram_WER'] = [torchmetrics.functional.word_error_rate(x['trigram_transcription'], 
    x['gloss']).item() for x in final_utts_to_score.to_dict('records')]

[NeMo W 2023-03-29 17:16:17 nemo_logging:349] /tmp/ipykernel_2743583/3962398152.py:1: SettingWithCopyWarning: 
    A value is trying to be set on a copy of a slice from a DataFrame.
    Try using .loc[row_indexer,col_indexer] = value instead
    
    See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
      final_utts_to_score['trigram_WER'] = [torchmetrics.functional.word_error_rate(x['trigram_transcription'],
    


In [54]:
final_utts_to_score.groupby('speaker_code')['neural_WER'].agg(np.mean)

speaker_code
CHI    0.980392
MOT    0.824529
Name: neural_WER, dtype: float64

In [55]:
final_utts_to_score.groupby('speaker_code')['trigram_WER'].agg(np.mean)

speaker_code
CHI    0.995098
MOT    0.804851
Name: trigram_WER, dtype: float64

# Todos

In [56]:
# [X] figure out why there are some with invalid duration specifications
# [X] subset the utterances to ones that finished
# [wontfix] transcribe_with_neural_rescoring only handles one file at a time
# [ ] get missing mp3s eg for Ethan
# [ ] Send WAVs through programmatic cleanup / preprocessing
# [ ] VAD and only detect things with speech. Strategy for dealing with silence in between speech vs. stripping
# [ ] get more details of CTC model from NEMO
# [ ] play with the alpha value -- upweight the ngram model